# Neural Networks

[Neural networks](https://en.wikipedia.org/wiki/Artificial_neural_network) are models inspired by biological brains for learning arbitrary functions.

# Function Approximation

To start with, let's train a neural network to learn a simple mathematical function such as `z = sin(x) + 2*y`:

In [1]:
import numpy as np
from sklearn import preprocessing
from sklearn.cross_validation import train_test_split
np.random.seed(0)

# Number of samples
n = 100

# Choose n random numbers for x and y
x = np.random.rand(n)
y = np.random.rand(n)

# Create an array of [x,y] scaled:
# We scale the data because neural networks perform better when all inputs are
# in a similar value range
data = preprocessing.scale(np.stack([x,y], axis=1))

# Create z.  We reshape it to an array of 1-element arrays for pyBrain
target = (np.sin(x) + 2*y).reshape(n,1)

# Create train/test split
data_train, data_test, target_train, target_test = train_test_split(
    data, target, test_size=.25, random_state=1
)

Now let's create a neural network:

In [2]:
from pybrain.tools.shortcuts import buildNetwork
from pybrain.structure import TanhLayer

network = buildNetwork(2, 5, 1, hiddenclass=TanhLayer)

The above creates a network with 2 inputs (x and y) has 5 hidden units and 1 output (z).

Now we need to create datasets that PyBrain can use.

In [3]:
from pybrain.datasets.classification import SupervisedDataSet

# Create a dataset with 2 inputs and 1 output
ds_train = SupervisedDataSet(2,1)

# add our data to the dataset
ds_train.setField('input', data_train)
ds_train.setField('target', target_train)

# Do the same for the test set
ds_test = SupervisedDataSet(2,1)
ds_test.setField('input', data_test)
ds_test.setField('target', target_test)

Finally, let's train our network and report it's accuracy:

In [4]:
from pybrain.supervised.trainers import BackpropTrainer
from pybrain.tools.validation import Validator

# Create a trainer for the network and training dataset
trainer = BackpropTrainer(network, ds_train)

# Train for a number of epochs and report accuracy:
for i in range(10):
    # Train 10 epochs
    trainer.trainEpochs(10)
    
    # Report mean squared error for training and testing sets
    # `network.activateOnDataset` will return the predicted values for each input in the dataset passed to it.
    # Then `Validator.MSE` returns the mean squared error of the returned value with the actual value.
    print("Train MSE:", Validator.MSE(network.activateOnDataset(ds_train), target_train))
    print("Test MSE:", Validator.MSE(network.activateOnDataset(ds_test), target_test))

Train MSE: 0.0470309778331
Test MSE: 0.0422927304237
Train MSE: 0.0351590448988
Test MSE: 0.0332700600733
Train MSE: 0.0284098213327
Test MSE: 0.0269475295626
Train MSE: 0.0240637906702
Test MSE: 0.0232706850741
Train MSE: 0.0205769872461
Test MSE: 0.0195560102946
Train MSE: 0.0179362664597
Test MSE: 0.0165537385038
Train MSE: 0.0151774137556
Test MSE: 0.0141666898109
Train MSE: 0.01297723041
Test MSE: 0.0116915662529
Train MSE: 0.0110174655498
Test MSE: 0.00985542496039
Train MSE: 0.00939671433821
Test MSE: 0.0082933702735


# Classification with Neural Networks

Neural networks can also be used for classification.

# Digits Sample Data

We'll be using the same dataset we used for the [classification notebook](classification.ipynb) - a dataset of 8x8 pixel handwritten digits such as the following:

![digits](../scikitlearn/digits.png)

The data is in the form of a 64 element array of integers representing grayscale values for each pixel.

In [5]:
from sklearn.datasets import load_digits

# Load all the samples for all digits 0-9
digits = load_digits()

# Assign the matrices to a variable `data`
data = digits.data

# Assign the labels to a variable `target`
target = digits.target.reshape((len(digits.target), 1))

# Split the data into 75% train, 25% test
data_train, data_test, target_train, target_test = train_test_split(
    data, target, test_size=.25, random_state=0
)

Now let's build our network again.  This time we have 64 inputs and 10 outputs (1 output for each digit).  We can also create multiple hidden layers.  For example two layers of eight hidden units each:

In [7]:
from pybrain.structure import SoftmaxLayer

# Create a network with 64 inputs, 2 layers of 16 hidden units and 10 outputs (for each digit 0-9)
network = buildNetwork(data.shape[1], 16, 16, 10, hiddenclass=SoftmaxLayer)

This time we'll use a dataset specifically for classification.  And we will create dummy variables for each of the labels instead of using the numerical value directly.

In [8]:
from pybrain.datasets.classification import ClassificationDataSet

# Create a dataset with 64 inputs
ds_train = ClassificationDataSet(data_train.shape[1])

# Set the input data
ds_train.setField('input', data_train)
ds_train.setField('target', target_train)

# The convert to dummy variables
# For instance, this will represent 0 as (1,0,0,0,0,0,0,0,0,0)
# 1 as (0,1,0,0,0,0,0,0,0,0), 4 as (0,0,0,0,1,0,0,0,0,0) and so on.
ds_train._convertToOneOfMany()

# Do the same for the test set
ds_test = ClassificationDataSet(data_test.shape[1])
ds_test.setField('input', data_test)
ds_test.setField('target', target_test)
ds_test._convertToOneOfMany()

Now we can train as we did before.  We'll need to train this network for longer because the model is more complicated as is the function.

In [9]:
from sklearn.metrics import accuracy_score

trainer = BackpropTrainer(network, ds_train)

for i in range(10):
    trainer.trainEpochs(50)
    
    # We use `argmax(1)` to convert the results back from the 10 outputs to a single label
    print("Training Accuracy:", accuracy_score(target_train, network.activateOnDataset(ds_train).argmax(1)))
    print("Testing Accuracy:", accuracy_score(target_test, network.activateOnDataset(ds_test).argmax(1)))

Training Accuracy: 0.749072011878
Testing Accuracy: 0.726666666667
Training Accuracy: 0.823311061618
Testing Accuracy: 0.771111111111
Training Accuracy: 0.856718634001
Testing Accuracy: 0.826666666667
Training Accuracy: 0.852264291017
Testing Accuracy: 0.831111111111
Training Accuracy: 0.842613214551
Testing Accuracy: 0.84
Training Accuracy: 0.853749072012
Testing Accuracy: 0.844444444444
Training Accuracy: 0.843355605048
Testing Accuracy: 0.851111111111
Training Accuracy: 0.839643652561
Testing Accuracy: 0.835555555556
Training Accuracy: 0.847067557535
Testing Accuracy: 0.833333333333
Training Accuracy: 0.855233853007
Testing Accuracy: 0.822222222222


# Further Reading

* [PyBrain Official Documentation](http://pybrain.org/docs/index.html)